#Training

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.pyplot as plt

In [ ]:
data_train = pd.read_csv("train.csv")
data_train.head()

In [ ]:
data_train.shape

Assigning each category a number for Category feature.

In [ ]:
category = {'A': 1,'B': 2,'C': 3,'D': 4,'E': 5,'F': 6,'G': 7,'H': 8}
data_train["category"] = data_train["category"].map(category)
data_train.head()

Removing Character 'F' present in data.

In [ ]:
data_train = data_train[data_train.views!='F']
data_train = data_train[data_train.likes!='F']
data_train = data_train[data_train.dislikes!='F']
data_train = data_train[data_train.comment!='F']
data_train.head()

Convert values to integers for views, likes, comments, dislikes and adview.

In [ ]:
data_train["views"] = pd.to_numeric(data_train["views"])
data_train["likes"] = pd.to_numeric(data_train["likes"])
data_train["dislikes"] = pd.to_numeric(data_train["dislikes"])
data_train["comment"] = pd.to_numeric(data_train["comment"])
data_train["adview"] = pd.to_numeric(data_train["adview"])
data_train.head()

In [ ]:
column_vidid = data_train["vidid"]

In [ ]:
from sklearn.preprocessing import LabelEncoder
data_train["duration"] = LabelEncoder().fit_transform(data_train["duration"])
data_train["vidid"] = LabelEncoder().fit_transform(data_train["vidid"])
data_train["published"] = LabelEncoder().fit_transform(data_train["published"])
data_train.head()

Convert time in seconds for duration

In [ ]:
import time
import datetime

In [ ]:
def duration_in_sec(x):
    y = x[2:]
    h = ''
    m = ''
    s = ''
    mm = ''
    P = ['H','M','S']
    for i in y:
        if i not in P:
            mm += i
        else:
            if(i == 'H'):
                h = mm;
                mm = ''
            elif(i == 'M'):
                m = mm;
                mm = ''
            else:
                s = mm
                mm = ''
    if(h == ''):
        h = '00'
    if(m == ''):
        m = '00'
    if(s == ''):
        s = '00'
    bp = h + ':' + m + ':' + s
    return bp
train=pd.read_csv("train.csv")
mp = pd.read_csv("train.csv")["duration"]
time = mp.apply(duration_in_sec)

In [ ]:
def func_sec(time_string):
    h, m, s = time_string.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

time1=time.apply(func_sec)

data_train["duration"]=time1
data_train.head()    

Visualization

Individual Plots

In [ ]:
plt.hist(data_train["category"])
plt.show()
plt.plot(data_train["adview"])
plt.show()

Remove videos with adview greater than 2000000 as outlier

In [ ]:
data_train = data_train[data_train["adview"] <2000000]

Heatmap

In [ ]:
import seaborn as sns
f, ax = plt.subplots(figsize=(10, 8))
corr = data_train.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
square=True, ax=ax,annot=True)
plt.show()


Split Data into train and test

In [ ]:
Y_train = pd.DataFrame(data = data_train.iloc[:, 1].values, columns = ['target'])
data_train=data_train.drop(["adview"],axis=1)
data_train=data_train.drop(["vidid"],axis=1)
data_train.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_train, Y_train, test_size=0.2, random_state=42)
X_train.shape

Normalise Data

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)
X_train.mean()

Evaluation Metrics

In [ ]:
from sklearn import metrics
def print_error(X_test, y_test, model_name):
    prediction = model_name.predict(X_test)
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, prediction))
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, prediction))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, prediction)))

Linear Regression


In [ ]:
from sklearn import linear_model
linear_regression = linear_model.LinearRegression()
linear_regression.fit(X_train, y_train)
print_error(X_test,y_test, linear_regression)

Support Vector Regressor


In [ ]:
from sklearn.svm import SVR
supportvector_regressor = SVR()
supportvector_regressor.fit(X_train,y_train)
print_error(X_test,y_test, supportvector_regressor)

Decision Tree Regressor


In [ ]:
from sklearn.tree import DecisionTreeRegressor
decision_tree = DecisionTreeRegressor()
decision_tree.fit(X_train, y_train)
print_error(X_test,y_test, decision_tree)

Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
n_estimators = 200
max_depth = 25
min_samples_split=15
min_samples_leaf=2
random_forest = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
random_forest.fit(X_train,y_train)
print_error(X_test,y_test, random_forest)

Artificial Neural Network


In [ ]:
import keras
from keras.layers import Dense
ann = keras.models.Sequential([Dense(6, activation="relu", input_shape=X_train.shape[1:]), Dense(6,activation="relu"), Dense(1)])
optimizer=keras.optimizers.Adam()
loss=keras.losses.mean_squared_error
ann.compile(optimizer=optimizer,loss=loss,metrics=["mean_squared_error"])
history=ann.fit(X_train,y_train,epochs=100)
ann.summary()
print_error(X_test,y_test,ann)

Saving Best Scikitlearn model


In [ ]:
import joblib
joblib.dump(supportvector_regressor, "supportvectorregressor_youtubeadview.pkl")

Saving Keras Artificial Neural Network model


In [ ]:
ann.save("ann_youtubeadview.h5")

# Testing

In [ ]:
data_test = pd.read_csv("test.csv")
data_test.head()

In [ ]:
data_test.shape

In [ ]:
model = joblib.load("/content/supportvectorregressor_youtubeadview.pkl")

In [ ]:
category = {'A': 1,'B': 2,'C': 3,'D': 4,'E': 5,'F': 6,'G': 7,'H': 8}
data_test["category"] = data_test["category"].map(category)
data_test.head()

In [ ]:
data_test = data_test[data_test.views!='F']
data_test = data_test[data_test.likes!='F']
data_test = data_test[data_test.dislikes!='F']
data_test = data_test[data_test.comment!='F']
data_test.head()

In [ ]:
data_test.shape

In [ ]:
data_test["views"] = pd.to_numeric(data_test["views"])
data_test["likes"] = pd.to_numeric(data_test["likes"])
data_test["dislikes"] = pd.to_numeric(data_test["dislikes"])
data_test["comment"] = pd.to_numeric(data_test["comment"])
data_test.head()

In [ ]:
c_vidid = data_test["vidid"]

In [ ]:
from sklearn.preprocessing import LabelEncoder
data_test["duration"] = LabelEncoder().fit_transform(data_test["duration"])
data_test["vidid"] = LabelEncoder().fit_transform(data_test["vidid"])
data_test["published"] = LabelEncoder().fit_transform(data_test["published"])
data_test.head()

In [ ]:
test=pd.read_csv("test.csv")
mp = pd.read_csv("test.csv")["duration"]
t = mp.apply(duration_in_sec)

In [ ]:
t1=t.apply(func_sec)

data_test["duration"]=t1
data_test.head()

In [ ]:
data_test = data_test.drop(["vidid"],axis=1)
data_test.head()

In [ ]:
scaler1 = MinMaxScaler()
x_test = data_test
x_test = scaler1.fit_transform(x_test)

In [ ]:
Prediction = model.predict(x_test)

In [ ]:
Prediction = pd.DataFrame(Prediction)
Prediction.info()

In [ ]:
Prediction = Prediction.rename(columns={0:"Adview"})
Prediction.head()

In [ ]:
Prediction.to_csv("PredictedAdview.csv")